# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234974635752                   -0.50    8.0         
  2   -7.250220719840       -1.82       -1.40    1.0   5.02ms
  3   -7.250877698475       -3.18       -1.86    1.0   5.08ms
  4   -7.250960573051       -4.08       -1.87    2.0   6.31ms
  5   -7.251330613211       -3.43       -2.65    1.0   5.16ms
  6   -7.251338003690       -5.13       -3.14    1.0   5.22ms
  7   -7.251338627939       -6.20       -3.44    2.0   6.29ms
  8   -7.251338782177       -6.81       -4.01    1.0   5.48ms
  9   -7.251338792656       -7.98       -4.14    3.0   7.20ms
 10   -7.251338798014       -8.27       -4.67    2.0   6.40ms
 11   -7.251338798674       -9.18       -5.55    2.0   6.64ms
 12   -7.251338798703      -10.54       -5.71    3.0   7.97ms
 13   -7.251338798704      -11.86       -6.11    1.0   5.79ms
 14   -7.251338798704      -12.32       -6.57    2.0   6.66ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.047073016769121075
[ Info: Arnoldi iteration step 2: normres = 0.7463992030941486
[ Info: Arnoldi iteration step 3: normres = 0.693594319732335
[ Info: Arnoldi iteration step 4: normres = 0.29289499163600663
[ Info: Arnoldi iteration step 5: normres = 0.3167519711440241
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.31e-02, 6.04e-02, 2.12e-01, 2.23e-01, 3.84e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4931213449423616
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.29e-03, 1.55e-01, 4.45e-01, 1.04e-01, 3.31e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09882351622650842
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.10e-04, 3.95e-02, 2.03e-02, 5.43e-02, 6.71e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11144086675834461
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.50e-